In [1]:
import numpy as np
import pandas as pd
import folium

In [2]:
wells = pd.read_csv("../data/wells/AllWells_20210915.csv")

# Oil & Gas, Dry Gas (no liquids), Gas and "Liquefied Gas"
wells_oilgas = wells[ (wells['WellType'] == 'OG')  |
                      (wells['WellType'] == 'DG')  |
                      (wells['WellType'] == 'GAS') |
                      (wells['WellType'] == 'LG') ]
wells_latlon = wells_oilgas[['Latitude', 'Longitude']].to_records(index=False)
wells_oilgas.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/wells/AllWells_20210915.csv'

In [ ]:
pooled_wells = pd.read_csv("../data/wells/pool_volumes.csv")
pwells_latlon = pooled_wells[['Latitude', 'Longitude']].to_records(index=False)
pwells_ids = pooled_wells['poolID'].tolist()
pwells_volumes = pooled_wells['totalco2'].tolist()
pooled_wells.head()

In [ ]:
emissions = pd.read_csv(r'../data/Emissions by Unit and Fuel Type - csv/UNIT_DATA-Table 1.csv')
emsCA2019 = emissions[emissions['Reporting Year'] == 2019]
facilities = pd.read_excel(r'../data/ghgp_data_2019.xlsx',
                          sheet_name='Direct Emitters', skiprows=3)
# One Michegan emitter lists its CA headquarters. Retag it as Michigan
facilities.loc[facilities['Facility Id'] == 1000594, 'State'] = 'MI'

facilities = facilities[(facilities.State == 'CA')]

facilitiesB = facilities.rename(inplace=False,
                                columns={'Facility Name':'facility_name',
                                         'City':'city',
                                         'Primary NAICS Code':'naics_code',
                                         'Industry Type (subparts)':'industry_subparts',
                                         'Industry Type (sectors)':'industry_sectors'})

emsCA2019loc = pd.merge(emsCA2019, facilitiesB[['Facility Id',
#                                               'Facility Name2',
                                               'city',
                                               'Zip Code',
                                               'Address',
                                               'County',
                                               'Latitude',
                                               'Longitude',
#                                               'Primary NAICS Code2',
                                               'industry_subparts',
                                               'industry_sectors']],
                        how='left', on='Facility Id')
emsCA2019locagg = emsCA2019loc[['Facility Id','Latitude','Longitude',
                                'Unit CO2 emissions (non-biogenic)']].dropna().groupby(
                                    'Facility Id').agg({'Latitude':'min',
                                                        'Longitude':'min',
                                                        'Unit CO2 emissions (non-biogenic)':'sum'})
emsCA2019locagg.reset_index(inplace=True)
emsCA2019locagg = emsCA2019locagg.rename(columns={'Facility Id':'facility_id',
                                                  'Latitude':'lat',
                                                  'Longitude':'lon',
                                                  'Unit CO2 emissions (non-biogenic)':'annual_emissions'})
emsCA2019locagg = emsCA2019locagg[emsCA2019locagg['annual_emissions'] > 0]
emsCA2019locagg.head()


In [3]:
pooled_wells.head()
#unique_suppliers


NameError: name 'pooled_wells' is not defined

In [4]:
#create map
#find unique suppliers for looping and filtering
unique_suppliers = emsCA2019locagg[['lat', 'lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers

#plot all the suppliers 
mapit = folium.Map( location=[37.3427, -119.2244], zoom_start=6)


#loop through suppliers, paint all the receivers in the same color
for i in range(0,unique_suppliers.shape[0]):
    
    #big circle for supplier
    folium.CircleMarker( location=[ unique_suppliers['lat'][i],unique_suppliers['lon'][i] ], fill=True, 
                        fill_color='blue',color='blue',  radius=1 ).add_to( mapit ) #'

#loop through suppliers, paint all the receivers in the same color
for i in range(0,pooled_wells.shape[0]):
    
    #big circle for supplier
    folium.CircleMarker( location=[ pooled_wells['Latitude'][i],pooled_wells['Longitude'][i] ], fill=True, 
                        fill_color='black',color='black',  radius=1 ).add_to( mapit ) #'

mapit

mapit.save(outfile= "all emitters.html")

NameError: name 'emsCA2019locagg' is not defined